In [1]:
import sys

sys.path = ["/u/hgaafer/pyiron/atomistics/"] + sys.path

import atomistics
atomistics

<module 'atomistics' from '/u/hgaafer/pyiron/atomistics/atomistics/__init__.py'>

In [2]:
from atomistics.workflows.elastic.helper import get_tasks_for_elastic_matrix
from atomistics.workflows import (
    optimize_positions_and_volume,
    LangevinWorkflow,
    get_hesse_matrix,
    get_thermal_properties_for_harmonic_approximation,
    get_tasks_for_harmonic_approximation,
    analyse_results_for_harmonic_approximation,
    get_tasks_for_quasi_harmonic_approximation,
    analyse_results_for_quasi_harmonic_approximation,
    get_thermal_properties_for_quasi_harmonic_approximation,
)

from atomistics.calculators import (
    evaluate_with_lammpslib_library_interface,
    evaluate_with_lammpslib,
    get_potential_by_name,
    evaluate_with_hessian,
    get_potential_by_name
)

from phonopy.units import VaspToTHz
from ase.build import bulk
import matplotlib.pyplot as plt

/u/hgaafer/pyiron/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)
/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/__init__.py:41: UserWarning: calc_molecular_dynamics_phonons_with_lammpslib() is not available as the import of the module named 'dynaphopy' failed.
  raise_warning(module_list=lammps_phonon_functions, import_error=e)
/u/hgaafer/pyiron/atomistics/atomistics/calculators/__init__.py:94: UserWarning: calc_molecular_dynamics_phonons_with_lammpslib() is not available as the import of the module named 'dynaphopy' failed.
  raise_warning(module_list=lammps_phonon_functions, import_error=e)


# QHA:

In [3]:
bulk_st = bulk("Cu", cubic= True)

In [9]:
pot_df = get_potential_by_name("2001--Mishin-Y--Cu-1--LAMMPS--ipr1" )

/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


In [60]:
task_dict, qh_dict = get_tasks_for_quasi_harmonic_approximation(
    structure=bulk_st,
    num_points=11,
    vol_range=0.05,
    interaction_range=10,
    factor=VaspToTHz,
    displacement=0.01,
)
task_dict

{'calc_energy': {0.95: Atoms(symbols='Cu108', pbc=True, cell=[10.646405210083579, 10.646405210083579, 10.646405210083579]),
  0.96: Atoms(symbols='Cu108', pbc=True, cell=[10.683630705999596, 10.683630705999596, 10.683630705999596]),
  0.97: Atoms(symbols='Cu108', pbc=True, cell=[10.72059858086807, 10.72059858086807, 10.72059858086807]),
  0.98: Atoms(symbols='Cu108', pbc=True, cell=[10.75731324614748, 10.75731324614748, 10.75731324614748]),
  0.99: Atoms(symbols='Cu108', pbc=True, cell=[10.79377899365842, 10.79377899365842, 10.79377899365842]),
  1.0: Atoms(symbols='Cu108', pbc=True, cell=[10.83, 10.83, 10.83]),
  1.01: Atoms(symbols='Cu108', pbc=True, cell=[10.865980330760825, 10.865980330760825, 10.865980330760825]),
  1.02: Atoms(symbols='Cu108', pbc=True, cell=[10.901723944536023, 10.901723944536023, 10.901723944536023]),
  1.03: Atoms(symbols='Cu108', pbc=True, cell=[10.937234696761076, 10.937234696761076, 10.937234696761076]),
  1.04: Atoms(symbols='Cu108', pbc=True, cell=[10.972

In [61]:
result_dict = evaluate_with_lammpslib(
    task_dict=task_dict,
    potential_dataframe = pot_df
)
result_dict

{'energy': {0.95: -380.575677011043,
  0.96: -381.1797365771137,
  0.97: -381.6487179986337,
  0.98: -381.98946223918733,
  0.99: -382.208173238575,
  1.0: -382.31048431356515,
  1.01: -382.30147765815695,
  1.02: -382.18499957149436,
  1.03: -381.9641686939824,
  1.04: -381.6422124906602,
  1.05: -381.2225202950284},
 'forces': {(0.95,
   0): array([[-9.10066060e-02, -8.59035065e-15, -8.54871729e-15],
         [-9.76030853e-04, -8.65973959e-15, -8.56259508e-15],
         [-9.69704473e-04, -9.55832635e-15, -9.52016244e-15],
         [ 7.36129067e-05,  2.88856132e-06, -8.53483950e-15],
         [-1.76090816e-04, -1.61820803e-04, -8.88178420e-15],
         [-1.71926281e-04,  1.56418267e-04, -9.33628175e-15],
         [ 7.36129067e-05, -2.88856131e-06, -9.59302082e-15],
         [-1.76090816e-04,  1.61820803e-04, -9.47505963e-15],
         [-1.71926281e-04, -1.56418267e-04, -1.05696701e-14],
         [ 7.36129067e-05, -8.46545056e-15,  2.88856132e-06],
         [-1.76090816e-04, -8.992806

In [5]:
task_dict, qh_dict = get_tasks_for_quasi_harmonic_approximation(
    structure=bulk_st,
    num_points=11,
    vol_range=0.05,
    interaction_range=10,
    factor=VaspToTHz,
    displacement=0.01,
)

result_dict = evaluate_with_lammpslib(
    task_dict=task_dict,
    potential_dataframe = pot_df
)

eng_internal_dict, phonopy_collect_dict = analyse_results_for_quasi_harmonic_approximation(
    qh_dict=qh_dict,
    output_dict=result_dict,
    dos_mesh=20,
    number_of_snapshots=None,
)

term_qh_dict = get_thermal_properties_for_quasi_harmonic_approximation(
    eng_internal_dict=eng_internal_dict,
    task_dict=task_dict,
    qh_dict=qh_dict,
    fit_type="polynomial",
    fit_order=3,
    t_min=1,
    t_max=200,
    t_step=20,
    temperatures=None,
    cutoff_frequency=None,
    pretend_real=False,
    band_indices=None,
    is_projection=False,
    quantum_mechanical=True,
)

term_qh_dict

/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: cmti001
--------------------------------------------------------------------------


{'temperatures': array([  1.,  21.,  41.,  61.,  81., 101., 121., 141., 161., 181., 201.]),
 'volumes': [47.50515232617367,
  47.50529066255871,
  47.50800419477625,
  47.517466981588925,
  47.53445194642627,
  47.55759437519629,
  47.585297803146744,
  47.616310302756176,
  47.649747041559664,
  47.684997757505144,
  47.721640204699014],
 'free_energy': array([0.12454683, 0.12450954, 0.12392528, 0.12170687, 0.11708739,
        0.10980353, 0.09987145, 0.08742239, 0.07262339, 0.05564346,
        0.03664067]),
 'entropy': array([7.25101354e-07, 7.10100267e-01, 5.76506564e+00, 1.57667490e+01,
        2.80899039e+01, 4.08467871e+01, 5.31684098e+01, 6.47379475e+01,
        7.54851579e+01, 8.54400029e+01, 9.46681718e+01]),
 'heat_capacity': array([8.47062224e-06, 2.32749682e+00, 1.62240798e+01, 3.52780701e+01,
        5.15001916e+01, 6.34530585e+01, 7.19482712e+01, 7.80034169e+01,
        8.23914764e+01, 8.56374271e+01, 8.80887688e+01])}

In [6]:
term_qh_dict = get_thermal_properties_for_quasi_harmonic_approximation(
    eng_internal_dict=eng_internal_dict,
    task_dict=task_dict,
    qh_dict=qh_dict,
    fit_type="polynomial",
    fit_order=3,
    t_min=1,
    t_max=800,
    t_step=50,
    temperatures=None,
    cutoff_frequency=None,
    pretend_real=False,
    band_indices=None,
    is_projection=False,
    quantum_mechanical=True,
)
term_qh_dict

{'temperatures': array([  1.,  51., 101., 151., 201., 251., 301., 351., 401., 451., 501.,
        551., 601., 651., 701., 751., 801.]),
 'volumes': [47.50515232617367,
  47.5117603081178,
  47.55759437519629,
  47.6327690849302,
  47.721640204699014,
  47.81759575014647,
  47.91784968874177,
  48.0211254163828,
  48.126810054091465,
  48.23461798467588,
  48.34444306864436,
  48.4562884118359,
  48.57023127908501,
  48.68640545262042,
  48.80499310102689,
  48.926222446742564,
  49.05036951938449],
 'free_energy': array([ 0.12454683,  0.12308222,  0.10980353,  0.08030591,  0.03664067,
        -0.01881467, -0.0842181 , -0.1581915 , -0.23968815, -0.32789468,
        -0.42216529, -0.52197757, -0.62690245, -0.73658322, -0.85072061,
        -0.96906198, -1.09139347]),
 'entropy': array([7.25101354e-07, 1.03088777e+01, 4.08467871e+01, 7.02138076e+01,
        9.46681718e+01, 1.15037566e+02, 1.32340922e+02, 1.47333546e+02,
        1.60545693e+02, 1.72353251e+02, 1.83028929e+02, 1.92775560e+02,

# Obtaining Elastic Constants (0K structure):

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$F(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

10.09.2025
- Free energy for different epsilon for a given lattice parameter (for each one I obtained from term_qh_dict['volume'] [no strains]) => get lattice parameter from the volumes
- Apply strain for each structure => how pot energy change? not only pot energy but look at free energy [apply strain in phonopy]

17.09.2025

1. elastic constants for a given structure (0K bulk structure) using pot energy
2. Do the same for isotropic strained structures on different temperatures but using free energy

func(structure, strains, energy) -> elastic constants

Raynol:

1. Take relaxed structure. Apply strains along xx, xy...etc. Generate elastic constants from total energy.
2. Do the same thing, this time with free energy instaed of total energy.
3. Now apply isotropic strains (aka 'volume expansion') and repeat step 2 and subsequently step 3.

In [17]:
bulk_st = bulk("Cu", cubic= True)

In [21]:
potential_dataframe = get_potential_by_name("2001--Mishin-Y--Cu-1--LAMMPS--ipr1" )

/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst


In [16]:
from atomistics.workflows.elastic.helper import get_tasks_for_elastic_matrix, analyse_results_for_elastic_matrix

In [51]:
task_dict, sym_dict= get_tasks_for_elastic_matrix(
    structure=bulk_st,
    eps_range = 0.005, # 0.005 
    num_of_point = 3, # 3 or 5
    sqrt_eta=True
)
task_dict, sym_dict

# unstrained structure is named "s_e_0".
# SGN: space group number.
# v0: equilibrium volume.
# LC: lattice class (cubic, hexagonal, etc.).
# Lag_strain_list: symmetry-reduced list of independent strain modes.
# epss: actual strain values in the range ±eps_range, with num_of_point sampling points.

/u/hgaafer/pyiron/atomistics/atomistics/workflows/elastic/symmetry.py:32: DeprecationWarning: dict interface is deprecated. Use attribute interface instead
  SGN = dataset["number"]


({'calc_energy': OrderedDict([('s_e_0',
                Atoms(symbols='Cu4', pbc=True, cell=[3.61, 3.61, 3.61])),
               ('s_01_e_m0_00500',
                Atoms(symbols='Cu4', pbc=True, cell=[3.5919046479549266, 3.5919046479549266, 3.5919046479549266])),
               ('s_01_e_0_00500',
                Atoms(symbols='Cu4', pbc=True, cell=[3.628005099224669, 3.628005099224669, 3.628005099224669])),
               ('s_08_e_m0_00500',
                Atoms(symbols='Cu4', pbc=True, cell=[3.5919046479549266, 3.5919046479549266, 3.61])),
               ('s_08_e_0_00500',
                Atoms(symbols='Cu4', pbc=True, cell=[3.628005099224669, 3.628005099224669, 3.61])),
               ('s_23_e_m0_00500',
                Atoms(symbols='Cu4', pbc=True, cell=[[3.6099092901893353, -0.018095809035334115, -0.018095809035334115], [-0.018095809035334115, 3.6099092901893353, -0.018095809035334115], [-0.018095809035334115, -0.018095809035334115, 3.6099092901893353]])),
               ('s_23_

In [52]:
result_dict = evaluate_with_lammpslib(
    task_dict=task_dict,
    potential_dataframe=potential_dataframe,
)
result_dict

{'energy': {'s_e_0': -14.159647567169056,
  's_01_e_m0_00500': -14.152389892719183,
  's_01_e_0_00500': -14.157631170918076,
  's_08_e_m0_00500': -14.155738842306423,
  's_08_e_0_00500': -14.159200296607338,
  's_23_e_m0_00500': -14.156233367938038,
  's_23_e_0_00500': -14.156218901602406}}

In [ ]:
# output_dict : output from lammps
fit_dict, sym_dict = analyse_results_for_elastic_matrix(output_dict = result_dict, sym_dict=sym_dict)
fit_dict, sym_dict

In [37]:
sym_dict

{'SGN': 225,
 'v0': 47.04588100000001,
 'LC': 'CI',
 'Lag_strain_list': ['01', '08', '23'],
 'epss': array([0.])}

In [49]:
def get_elastic_constants(structure: "ase.atoms.Atoms", eps_range : float = 0.005, pot: str = "2001--Mishin-Y--Cu-1--LAMMPS--ipr1"):

    from atomistics.workflows.elastic.helper import get_tasks_for_elastic_matrix, analyse_results_for_elastic_matrix

    task_dict, sym_dict= get_tasks_for_elastic_matrix(
        structure=bulk_st,
        eps_range = 0.005,  
        num_of_point = 3,
        sqrt_eta=True
    )
    
    potential_dataframe = get_potential_by_name(pot)
    
    result_dict = evaluate_with_lammpslib(
        task_dict=task_dict,
        potential_dataframe=potential_dataframe,
    )
    
    # only meaningful values are C11, C12, C44 for cubic metals in 'elastic_matrix'
    fit_dict, sym_dict = analyse_results_for_elastic_matrix(output_dict = result_dict, sym_dict=sym_dict)

    return fit_dict

In [53]:
fit_dict = get_elastic_constants(structure=bulk_st)

/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/potential.py:324: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/u/hgaafer/pyiron/atomistics/atomistics/calculators/lammps/helpers.py:29: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  lmp.interactive_structure_setter(


In [54]:
fit_dict
# only meaningful values are C11, C12, C44 for cubic metals in 'elastic_matrix'

{'elastic_matrix': array([[172.27219477, 124.42005281, 124.42005281,   0.        ,
           0.        ,   0.        ],
        [124.42005281, 172.27219477, 124.42005281,   0.        ,
           0.        ,   0.        ],
        [124.42005281, 124.42005281, 172.27219477,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,  77.67933384,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
          77.67933384,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,  77.67933384]]),
 'elastic_matrix_inverse': array([[ 0.01472336, -0.00617435, -0.00617435,  0.        ,  0.        ,
          0.        ],
        [-0.00617435,  0.01472336, -0.00617435,  0.        ,  0.        ,
          0.        ],
        [-0.00617435, -0.00617435,  0.01472336,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.       

In [58]:
print(fit_dict['elastic_matrix'])

[[172.27219477 124.42005281 124.42005281   0.           0.
    0.        ]
 [124.42005281 172.27219477 124.42005281   0.           0.
    0.        ]
 [124.42005281 124.42005281 172.27219477   0.           0.
    0.        ]
 [  0.           0.           0.          77.67933384   0.
    0.        ]
 [  0.           0.           0.           0.          77.67933384
    0.        ]
 [  0.           0.           0.           0.           0.
   77.67933384]]
